# Projet TAL - RI
_Charlotte Kruzic & Zoé Marquis_
# Étape 1

In [ ]:
!pip install pandas unidecode

In [ ]:

# Imports nécessaires

%matplotlib inline

# Suppression de l'affichage des messages d'avertissement
import warnings
warnings.filterwarnings('ignore')

import string
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import metrics
from sklearn import model_selection
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from sklearn import model_selection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
import seaborn as sns
sns.set_theme(style="darkgrid")

set_config(display='diagram')

nltk.download('stopwords')
nltk.download('punkt')

### Charger les données & exploration

In [ ]:
df_train = pd.read_csv("train.tsv", sep='\t')

In [ ]:
df_train.head()

In [ ]:
print(df_train.columns)

### Nettoyer les données 

In [ ]:
# identifier les valeurs manquantes
valeurs_manquantes = df_train.isnull().sum()
print(valeurs_manquantes) 
# pas de valeur manquante

In [ ]:
# identifier les lignes dupliquées
dupliquees = df_train.duplicated()
print(dupliquees.sum())
# 1 ligne dupliquée

In [ ]:
# supprimer les lignes dupliquées
df_train = df_train.drop_duplicates()
dupliquees = df_train.duplicated()
print(dupliquees.sum())

In [ ]:
print(df_train.info())

Le jeu de données contient 1475 lignes.  
Toutes les colonnes contiennent du texte (donc il n'est pas nécessaire de traiter les données numériques).   
On s'intéresse seulement aux colonnes `headline` et `text`.

### Préparer les données pour l'apprentissage de la catégorie
à partir du titre et du texte uniquement

Séparation des données de la catégorie à prédire

In [ ]:
X = df_train[['headline', 'text']]
y = df_train.category

In [ ]:
X.head()

In [ ]:
# afficher toutes les catégories
y.unique()

### Prétraitements des données

- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides

In [ ]:
# def split_into_tokens_nltk(desc) :
#     return word_tokenize(desc, language='french')
# 
# # Liste des mots vides de NLTK + signes de ponctuation
# nltk_stopwords = stopwords.words('french')+list(string.punctuation)
# # +['``'] : à essayer


# Liste des mots vides de NLTK + signes de ponctuation
nltk_stopwords = set(stopwords.words('french')) | set(string.punctuation) 

# Fonction de tokenisation personnalisée
def split_into_tokens_nltk(desc):
    tokens = word_tokenize(desc, language='french')
    tokens = [token.lower() for token in tokens if token.lower() not in nltk_stopwords]
    return tokens

On va essayer plusieurs prétraitements différents supplémentaires dont : 
- Lemmatisation
- Désuffixation
- N-grammes

In [ ]:
stemmer = SnowballStemmer(language='french')

def stemming(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

def stemming_transformer(X):
    return X.apply(stemming)

In [ ]:
# def lemmatize_text(text):
#     # modèle français de Spacy
#     nlp = spacy.load("fr_core_news_sm")
#     lemmatized_text = ' '.join([token.lemma_ for token in nlp(text)])
#     return lemmatized_text

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text, language='french')
    lemmatized_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_text

def lemmatization_transformer(X):
    return X.apply(lemmatize_text)

On va essayer 3 types de traits : 
- TF-IDF (avec différentes fréquences)
- Sac de mots (avec différentes fréquences)
- Informations statistiques : longueur en nombre de caractères et nombre approximatif de phrases

In [ ]:
def text_stats(descriptions):
    return [{"length": len(text), "num_sentences": text.count(".")}
            for text in descriptions]
    
text_stats_transformer = FunctionTransformer(text_stats)
text_stats_vectorizer = DictVectorizer(sparse=False)
min_max_scaler = MinMaxScaler()    

In [ ]:
tfidf_001 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.01)
bow_001 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                          # stop_words=nltk_stopwords,
                          min_df=0.01)

tfidf_001_ngram2 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.01,
                            ngram_range=(1, 2)
                            )
bow_001_ngram2 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                          # stop_words=nltk_stopwords,
                          ngram_range=(1, 2),
                          min_df=0.01)

tfidf_001_ngram3 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.01,
                            ngram_range=(1, 3)
                            )
bow_001_ngram3 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                          # stop_words=nltk_stopwords,
                          ngram_range=(1, 3),
                          min_df=0.01)

tfidf_001_ngram4 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.01,
                            ngram_range=(1, 4)
                            )
bow_001_ngram4 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                          # stop_words=nltk_stopwords,
                          ngram_range=(1, 4),
                          min_df=0.01)

tfidf_010 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.1)
bow_010 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                          # stop_words=nltk_stopwords,
                          min_df=0.1)

tfidf_005 = TfidfVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                            lowercase=True, 
                            # stop_words=nltk_stopwords,
                            min_df = 0.05)
bow_005 = CountVectorizer(tokenizer=split_into_tokens_nltk,
                            token_pattern=None,
                          lowercase=True,
                         #  stop_words=nltk_stopwords,
                          min_df=0.05)

### Modèles

In [ ]:
models = [
    ('Baseline', DummyClassifier(strategy='most_frequent')),
    ('Mutinomial NB', MultinomialNB()),
    ('CART', DecisionTreeClassifier()),
    ('LR', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('Random forest', RandomForestClassifier()),
    ('SVM', SVC(kernel='rbf')),
]

### Évaluation

Validation croisée à 5 plis

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,
                                                    random_state=3, shuffle=True)

In [ ]:
stratkfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

In [ ]:
def evaluate(modele, trait):
    model_pipeline = make_pipeline(trait, modele)
    y_pred = model_selection.cross_val_predict(model_pipeline, X_train, y_train,
                                               cv=stratkfold, n_jobs=-1)
    return y_pred

In [ ]:
def run(chaines, modeles):
    classification_reports = {}

    for identifiant, colonne_transformateur in chaines:
        classification_reports[identifiant] = {}
        
        for name, model in models :
            y_pred = evaluate(model, trait=colonne_transformateur)
            
            report = classification_report(y_train, y_pred, output_dict=True)
            
            classification_reports[identifiant][name] = {
                "accuracy" : report["accuracy"],
                "f1-score" : report["macro avg"]["f1-score"],
                "precision" : report["macro avg"]["precision"],
                "recall" : report["macro avg"]["recall"],
            }
        
    return classification_reports

### Visualiser pour comparer

In [ ]:
def afficher_chaines(reports):
    for chaine_num, chaine_data in reports.items():
        fig, axs = plt.subplots(2, 2, figsize=(8, 6))
        fig.suptitle(chaine_num) 

        metrics = ['accuracy', 'f1-score', 'precision', 'recall']

        for ax, metric in zip(axs.flatten(), metrics):
            model_names = list(chaine_data.keys())
            metric_values = [model_data[metric] for model_data in chaine_data.values()]
            
            
            mean_metric = np.mean(metric_values)
            max_metric = np.max(metric_values)
            
            ax.axvline(mean_metric, color='r', linestyle='--', label=f'Mean ({mean_metric:.2f})')
            ax.axvline(max_metric, color='g', linestyle='--', label=f'Max ({max_metric:.2f})')
            

            ax.barh(model_names, metric_values, color='b')
            ax.set_title(metric.upper()) 
            ax.set_xlabel(metric.capitalize())  
            ax.set_ylabel('Modèles')  
            ax.invert_yaxis()  
            ax.set_xlim(0, 1)
            
            ax.legend()
            
        plt.tight_layout()
        plt.show()

In [ ]:
def comparer_chaines(reports):
    metrics = ['accuracy', 'f1-score', 'precision', 'recall']
    metric_values = {metric: [] for metric in metrics}

    for chaine_num, chaine_data in reports.items():
        longueur = len(chaine_data.keys())
        for model, values in chaine_data.items():
            for metric in metrics:
                metric_values[metric].append((chaine_num, model, values[metric]))
                

    for metric in metrics:
        fig, axs = plt.subplots(2, 2, figsize=(8, 6))
        fig.suptitle(f'Comparaison de la métrique {metric.capitalize()} par modèle')  # Titre de la figure

        for i, ax in enumerate(axs.flat):
            values = metric_values[metric][i*longueur:(i+1)*longueur]

            chaine_names = [value[0] for value in values]
            model_names = [value[1] for value in values]
            metric_scores = [value[2] for value in values]
            
            mean_metric_score = np.mean(metric_scores)
            max_metric_score = np.max(metric_scores)
            
            ax.axvline(mean_metric_score, color='r', linestyle='--', label=f'Moyenne ({mean_metric_score:.2f})')
            ax.axvline(max_metric_score, color='g', linestyle='--', label=f'Maximum ({max_metric_score:.2f})')

            ax.barh(model_names, metric_scores, color='b')
            ax.set_title(chaine_names[0])  
            ax.set_xlabel(chaine_names[0]) 
            ax.set_ylabel('Chaines')  
            ax.invert_yaxis() 
            ax.set_xlim(0, 1)
            ax.legend()
            
        plt.tight_layout()

    plt.show()


## Premières chaines

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides

Et on utilise les traits suivants :

| n° | headline | text |
| --- | --- | --- |
| 1 | tf-idf (fréquence = 0.01) | tf-idf (fréquence = 0.01) | 
| 2 | sac de mots (fréquence = 0.01) | sac de mots (fréquence = 0.01) | 
| 3 | tf-idf (fréquence = 0.01) |  sac de mots (fréquence = 0.01) | 
| 4 | sac de mots (fréquence = 0.01) |  tf-idf (fréquence = 0.01) | 

In [ ]:
chaines = [
    ('chain1', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_tfidf_001', tfidf_001, 'text'),
        ]
    )),
    
    ('chain2', ColumnTransformer(
        [
            ('headline_bow_001', bow_001, 'headline'),
            ('text_bow_001', bow_001, 'text'),
        ]
    )),
    
    ('chain3', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
        ]
    )),
    
    ('chain4', ColumnTransformer(
        [
            ('headline_bow_001', bow_001, 'headline'),
            ('text_tfidf_001', tfidf_001, 'text'),
        ]
    )),
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
afficher_chaines(classification_reports)

On s'aperçoit tout de suite que les 4 meilleurs modèles sont (presque) à chaque fois 
- Multinomial NB
- LR (Régression Logistique)
- Random forest
- SVM

Nous redéfinissons donc la liste des modèles que nous utiliserons : 

In [ ]:
models = [
    ('Mutinomial NB', MultinomialNB()),
    ('LR', LogisticRegression()),
    ('Random forest', RandomForestClassifier()),
    ('SVM', SVC(kernel='rbf')),
]

In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

On élimine les chaines 1 et 4 car elles ont un F1-score moins bons.

Essayons maintenant d'ajouter les traits d'informations statistiques.

## Deuxièmes chaines

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides

Et on utilise les traits suivants :

| n° | headline | text |
| --- | --- | --- |
| 2 | sac de mots (fréquence = 0.01) | sac de mots (fréquence = 0.01) | 
| 3 | tf-idf (fréquence = 0.01) |  sac de mots (fréquence = 0.01) | 
| 5 | sac de mots (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 
| 6 | tf-idf (fréquence = 0.01) |  sac de mots (fréquence = 0.01) + informations statistiques | 

In [ ]:
chaines = [
    ('chain2', ColumnTransformer(
        [
            ('headline_bow_001', bow_001, 'headline'),
            ('text_bow_001', bow_001, 'text'),
        ]
    )),
    
    ('chain3', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
        ]
    )),
    
    ('chain5', ColumnTransformer(
        [
            ('headline_bow_001', bow_001, 'headline'),
            ('text_bow_001', bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain6', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

Les mesures sont peu impactées par ce changement de traits.  
On garde la chaine 6 pour la suite car elle est légèrement meilleure lorsqu'il s'agit de Precision Rappel (pour la valeur maximale).   
On supprime aussi le modèle SVM qui n'est pas le plus performant.

In [ ]:
models = [
    ('Mutinomial NB', MultinomialNB()),
    ('LR', LogisticRegression()),
    ('Random forest', RandomForestClassifier()),
]

## Troisièmes chaines

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides
- **N-grammes**

Et on utilise les traits suivants :

| n° | headline | text | n-gramme |
| --- | --- | --- | --- |
| 6 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 
| 7 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 2 |
| 8 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 3 |
| 9 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 4 |

In [ ]:
chaines = [
    ('chain6', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain7', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram2, 'headline'),
            ('text_bow_001',  bow_001_ngram2, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain8', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram3, 'headline'),
            ('text_bow_001',  bow_001_ngram3, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain9', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram4, 'headline'),
            ('text_bow_001',  bow_001_ngram4, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

On garde la chaine 6 qui a les mêmes mesures que la chaine 9, supérieures aux autres chaines en terme de Précision, identiques pour les autres mesures.

On essaye les n-grammes lorsq'uon utilise TF-IDF pour le texte.

In [ ]:
chaines = [
    ('chain6', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain7tfidf', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram2, 'headline'),
            ('text_bow_001',  tfidf_001_ngram2, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain8tfidf', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram3, 'headline'),
            ('text_bow_001',  tfidf_001_ngram3, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain9tfidf', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001_ngram4, 'headline'),
            ('text_bow_001',  tfidf_001_ngram4, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

On garde la chaine 6 : l'*accuracy* est identique mais le F1-score est moins bon.

## Quatrièmes chaines 

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides
- **Fréquences** 

Et on utilise les traits suivants :

| n° | headline | text | 
| --- | --- | --- | 
| 6 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 
| 10 | tf-idf (fréquence = 0.1) | sac de mots (fréquence = 0.1) + informations statistiques | 
| 11 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.1) + informations statistiques | 
| 12 | tf-idf (fréquence = 0.1) | sac de mots (fréquence = 0.01) + informations statistiques | 

In [ ]:
chaines = [
    ('chain6', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain10', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  bow_010, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain11', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_010, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain12', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

On élimine les chaines 10 et 11 et on essaye une autre valeur pour la fréquence.

## Cinquièmes chaines 

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides
- **Fréquences** 

Et on utilise les traits suivants :

| n° | headline | text | 
| --- | --- | --- | 
| 6 | tf-idf (fréquence = 0.01) | sac de mots (fréquence = 0.01) + informations statistiques | 
| 12 | tf-idf (fréquence = 0.1) | sac de mots (fréquence = 0.01) + informations statistiques | 
| 13 | tf-idf (fréquence = 0.05) | sac de mots (fréquence = 0.05) + informations statistiques | 
| 14 | tf-idf (fréquence = 0.1) | sac de mots (fréquence = 0.05) + informations statistiques | 

In [ ]:
chaines = [
    ('chain6', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_001, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain12', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    
    ('chain13', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_005, 'headline'),
            ('text_bow_001',  bow_005, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain14', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  bow_005, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

On conserve la chaine 12 qui obtient les mêmes scores pour toutes les mesures, et est légèrement supérieure en terme de Rappel.

## Sixièmes chaines 

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides
- **Désuffixation**

Et on utilise les traits suivants :

| n° | headline | text | 
| --- | --- | --- | 


In [ ]:
chaines = [
    ('chain12', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    ('chain15', ColumnTransformer(
        [
            ('headline_tfidf_001', 
                Pipeline(
                 [
                     ('stemming', FunctionTransformer(stemming_transformer)),
                     ('text_stats', tfidf_010),
                 ]
                ),
                 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    ('chain16', ColumnTransformer(
        [
            ('headline_tfidf_001', tfidf_010, 'headline'),
            ('text_bow_001',  
                Pipeline(
                 [
                     ('stemming', FunctionTransformer(stemming_transformer)),
                     ('text_stats', bow_001),
                 ]
                ),
                 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    ('chain17', ColumnTransformer(
        [
            ('headline_tfidf_001', 
                Pipeline(
                 [
                     ('stemming', FunctionTransformer(stemming_transformer)),
                     ('text_stats', tfidf_010),
                 ]
                ),
                 'headline'),
            ('text_bow_001',  
                Pipeline(
                 [
                     ('stemming', FunctionTransformer(stemming_transformer)),
                     ('text_stats', bow_001),
                 ]
                ),
                 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

Les chaines 12 et 15 sont légèrement meilleures en terme de précision, on conserve la 12.

## Septièmes chaines 

Le pré-traitement appliqué est :
- Nettoyage du texte : supprimer les caractères spéciaux, les ponctuations, les accents convertir en minuscules
- Tokenisation
- Suppression des mots vides
- Désuffixation
- Lemmatisation

Et on utilise les traits suivants :

| n° | headline | text | 
| --- | --- | --- | 
| 12 |

In [ ]:
chaines = [
    ('chain12', ColumnTransformer(
        [
            ('headline_tfidf_010', tfidf_010, 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain18', ColumnTransformer(
        [
            ('headline_tfidf_010', 
                Pipeline(
                 [
                     ('lemmatization',  FunctionTransformer(lemmatization_transformer)),
                     ('text_stats', tfidf_010),
                 ]
                ),
                 'headline'),
            ('text_bow_001',  bow_001, 'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain19', ColumnTransformer(
        [
            ('headline_tfidf_010', tfidf_010, 'headline'),
            ('text_bow_001',  
              Pipeline(
                 [
                     ('lemmatization',  FunctionTransformer(lemmatization_transformer)),
                     ('text_stats', bow_001),
                 ]
                ),
              'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
    
    ('chain20', ColumnTransformer(
        [
            ('headline_tfidf_010', 
                Pipeline(
                 [
                     ('lemmatization',  FunctionTransformer(lemmatization_transformer)),
                     ('text_stats', tfidf_010),
                 ]
                ),
                 'headline'),
            ('text_bow_001',  
              Pipeline(
                 [
                     ('lemmatization',  FunctionTransformer(lemmatization_transformer)),
                     ('text_stats', bow_001),
                 ]
                ),
              'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    )),
]


In [ ]:
classification_reports = run(chaines, models)

In [ ]:
comparer_chaines(classification_reports)

Finalement, on conserve le modèle Multinomial NB car il nous permet de sélectionner l'accuracy et le F1-score maximaux.

Inspection des valeurs pour les 4 chaines :

In [ ]:
def round_dict_values(d):
    return {k: round(v, 3) for k, v in d.items()}

print("12",classification_reports["chain12"]["Mutinomial NB"])
print("18",classification_reports["chain18"]["Mutinomial NB"])
print("19",classification_reports["chain19"]["Mutinomial NB"])
print("20",classification_reports["chain20"]["Mutinomial NB"])


On sélectionne donc finalement la chaine 19 (scores égaux à 20 mais 20 lemmatise en plus le titre, ce que ne fait pas 19).

La combinaison sélectionnée est donc 
- modèle = Multinomial NB
- traits = 
  - TF-IDF, fréquence 0.1 pour le titre
    - Prétraitement :
      - Nettoyage du texte
      - Tokenisation
      - Suppression des mots vides
  - Sac de mots, fréquence 0.01 pour le texte
    - Prétraitement : 
      - Nettoyage du texte
      - Tokenisation
      - Suppression des mots vides
      - Lemmatisation
  - Informations statistiques sur le texte
    - Longueur du texte en nombre de caractères, nombre de phrases

----

## BiLSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sns.set_theme(style="darkgrid")

### Chargement des données

Les dossiers suivant contiennent les vecteurs de mots pré-entrainés utilisés, ils doivent être téléchargés et dézippés :
- https://nlp.stanford.edu/data/glove.6B.zip
- https://nlp.stanford.edu/data/glove.42B.300d.zip

#### Corpus annoté

In [ ]:
# Lecture du fichier
df = pd.read_csv("train.tsv", sep="\t")

In [ ]:
# Liste des classes
class_names = sorted(df['category'].unique())
print("Classes :", class_names)
print("Nombre d'exemplaires :", len(df))

In [ ]:
# On associe à chaque classe un identifiant unique
class_index = {class_names[i]:i for i in range(len(class_names))}
class_index

In [ ]:
X_train = df["headline"] + ' ' + df["text"]
#Remplacement des noms des classes par un entier
y_train = df["category"].map(class_index)

In [ ]:
def get_vectorizer(documents, max_voc_size=8000, max_seq_length=50, batch_size=128):
  vectorizer = TextVectorization(max_tokens=max_voc_size,
                                 output_sequence_length=max_seq_length)
  text_ds = tf.data.Dataset.from_tensor_slices(documents).batch(batch_size)
  vectorizer.adapt(text_ds)
  return vectorizer

In [ ]:
keras_vectorizer = get_vectorizer(X_train)

In [ ]:
voc = keras_vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

#### Plongements de mots pré-entraînés


In [ ]:
def load_embeddings(embeddings_file):
  embeddings_index = {}
  with open(embeddings_file, 'r', encoding='utf8') as f:
      for line in f:
          word, coefs = line.split(maxsplit=1)
          coefs = np.fromstring(coefs, "f", sep=" ")
          embeddings_index[word] = coefs
  print(f'{len(embeddings_index)} vecteurs de mots ont été lus')
  return embeddings_index

In [ ]:
# Chargement des plongements du fichier Glove
glove_embeddings = load_embeddings('glove.42B.300d/glove.42B.300d.txt')
#glove_embeddings = load_embeddings('glove.6B/glove.6B.300d.txt')

In [ ]:
def get_embedding_matrix(vocabulary, embeddings_index, embedding_dim = 300):
  num_tokens = len(vocabulary)
  hits = 0
  misses = 0

  embedding_matrix = np.zeros((num_tokens, embedding_dim))
  for word, i in word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
          hits += 1
      else:
          misses += 1
  print(f'{hits} mots ont été trouvés dans les plongements pré-entraînés')
  print(f'{misses} sont absents')
  return embedding_matrix

In [ ]:
# Construction de la matrice de plongements à partir du vocabulaire
glove_embedding_matrix = get_embedding_matrix(voc, glove_embeddings)

#glove.42B.300d/glove.42B.300d.txt
#6147 mots ont été trouvés dans les plongements pré-entraînés
#1853 sont absents

#glove.6B/glove.6B.300d.txt
#3619 mots ont été trouvés dans les plongements pré-entraînés
#4381 sont absents


### Construction et entraînement du modèle

In [ ]:
def get_biLSTM_model(voc_size, embedding_matrix, embedding_dim=300):
  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedding_layer = Embedding(voc_size, embedding_dim, trainable=True,
      embeddings_initializer=keras.initializers.Constant(embedding_matrix),
  )

  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))(embedded_sequences)
  preds = layers.Dense(len(class_names), activation="softmax")(x)
  model = keras.Model(int_sequences_input, preds)
  return model

In [ ]:
# Affichage de l'architecture du modèle
biLSTM_model = get_biLSTM_model(len(voc), glove_embedding_matrix)
biLSTM_model.summary()

In [ ]:
# Fonction pour l'entraînement d'un modèle
def train_model(X, y, model_function, vectorizer,
                voc_size, embedding_matrix, embedding_dim=300, batch_size=128):
  acc_per_fold = []
  loss_per_fold = []
  histories = []
  folds = 5
  stratkfold = model_selection.StratifiedKFold(n_splits=folds, shuffle=True,
                                              random_state=12)
  fold_no = 1
  for train, test in stratkfold.split(X, y):
    m_function = globals()[model_function]
    model = m_function(voc_size, embedding_matrix, embedding_dim)

    print('------------------------------------------------------------------------')
    print(f'Entraînement pour le pli {fold_no} ...')
    fold_x_train = vectorizer(X.iloc[train].to_numpy()).numpy()
    fold_x_val = vectorizer(X.iloc[test].to_numpy()).numpy()
    fold_y_train = y.iloc[train].to_numpy()
    fold_y_val = y.iloc[test].to_numpy()
    
    model.compile(
      loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
    )
    # Entraînement
    history = model.fit(fold_x_train, fold_y_train, batch_size=batch_size,
                        epochs=10, validation_data=(fold_x_val, fold_y_val))
    histories.append(history)
    # Evaluation sur les données de validation
    scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
    print(f'Scores pour le pli {fold_no}: {model.metrics_names[0]} = {scores[0]:.2f};',
          f'{model.metrics_names[1]} = {scores[1]*100:.2f}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    fold_no = fold_no + 1

  # Affichage des scores moyens par pli
  print('---------------------------------------------------------------------')
  print('Scores par pli')
  for i in range(0, len(acc_per_fold)):
    print('---------------------------------------------------------------------')
    print(f'> Pli {i+1} - Loss: {loss_per_fold[i]:.2f}',
          f'- Accuracy: {acc_per_fold[i]:.2f}%')
  print('---------------------------------------------------------------------')
  print('Scores moyens pour tous les plis :')
  print(f'> Accuracy: {np.mean(acc_per_fold):.2f}',
        f'(+- {np.std(acc_per_fold):.2f})')
  print(f'> Loss: {np.mean(loss_per_fold):.2f}')
  print('---------------------------------------------------------------------')
  return histories

In [ ]:
#Séparation données
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Entraînement du modèle
biLSTM_histories = train_model(X_train, y_train, 'get_biLSTM_model',
                            keras_vectorizer, len(voc), glove_embedding_matrix)

In [ ]:
print(X_train, X_test, y_train, y_test)

In [ ]:
def plot_results(histories):
  accuracy_data = []
  loss_data = []
  for i, h in enumerate(histories):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']
    for j in range(len(acc)):
      accuracy_data.append([i+1, j+1, acc[j], 'Entraînement'])
      accuracy_data.append([i+1, j+1, val_acc[j], 'Validation'])
      loss_data.append([i+1, j+1, loss[j], 'Entraînement'])
      loss_data.append([i+1, j+1, val_loss[j], 'Validation'])

  acc_df = pd.DataFrame(accuracy_data,
                        columns=['Pli', 'Epoch', 'Accuracy', 'Données'])
  sns.relplot(data=acc_df, x='Epoch', y='Accuracy', hue='Pli', style='Données',
              kind='line')

  loss_df = pd.DataFrame(loss_data, columns=['Pli', 'Epoch', 'Perte', 'Données'])
  sns.relplot(data=loss_df, x='Epoch', y='Perte', hue='Pli', style='Données',
              kind='line')

In [ ]:
plot_results(biLSTM_histories)

### Prédictions avec le modèle

In [ ]:
X_test_vectorized = keras_vectorizer(X_test.to_numpy()).numpy()
y_pred_prob = biLSTM_model.predict(X_test_vectorized)
y_pred_indices = np.argmax(y_pred_prob, axis=1)

In [ ]:
# Rapport de classification
report = classification_report(y_test, y_pred_indices, target_names=class_names)
print(report)

## Analyse

## Application du meilleur modèle aux données de test

In [ ]:
#Chargement du jeu de données de test
df_test = pd.read_csv("test.tsv", sep='\t')


In [ ]:
('chain19', ColumnTransformer(
        [
            ('headline_tfidf_010', tfidf_010, 'headline'),
            ('text_bow_001',  
              Pipeline(
                 [
                     ('lemmatization',  FunctionTransformer(lemmatization_transformer)),
                     ('text_stats', bow_001),
                 ]
                ),
              'text'),
            (
             'text_stats',
             Pipeline(
                 [
                     ('text_stats', text_stats_transformer),
                     ('vect', text_stats_vectorizer),
                     ('scaling', min_max_scaler)
                 ]
             ),
             'text'
         )
        ]
    ))